A Jupyter Notebook script for scraping data from the NCBI website
    
## Install

In [ ]:
!pip install  BeautifulSoup

In [ ]:
!pip install requests

In [ ]:
!pip install  pandas

In [ ]:
!pip install  tqdm

## Import

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm


In [4]:

tqdm.pandas()
def find_submited_by(acc_arg):
    Answer = dict()
    ACC  = str(acc_arg)
    URL = f'https://www.ncbi.nlm.nih.gov/sra/?term={ACC}'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    div_results = soup.find_all('div',class_='sra-full-data')
    for num, i in enumerate(div_results):
        string=i.getText()
        string = string.replace(u'\xa0', u' ')
        if "Submitted by:" in string:
            Answer['ACC'] = ACC
            Answer['Submitted_by'] = string.split(':')[1]
            return string.split(':')[1]
def find_ID(acc_arg):
    Answer = dict()
    ACC  = str(acc_arg)
    URL = f'https://www.ncbi.nlm.nih.gov/sra/?term={ACC}'
    page = requests.get(URL)    
    soup1 = BeautifulSoup(page.content, 'html.parser') 
    div_results2 = soup1.find_all('div',class_='resc')
    for num, i in enumerate(div_results2):
        string=i.getText()
        string = string.replace(u'\xa0', u' ')
        if "ID:" in string:
            Answer['ACC'] = ACC
            Answer['ID'] = string.split(':')[1]
            return string.split(':')[1]
def find_submission_biosample_field(ID_arg):
    Answer = dict()
    ID     = str(ID_arg)
    URL   = f'https://www.ncbi.nlm.nih.gov/biosample?LinkName=sra_biosample&from_uid={ID}'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')  
    div_results = soup.find_all('div',class_='docsum')
    for num, i in enumerate(div_results):
        string=i.getText()
        string = string.replace(u'\xa0', u'')
        if "Submission" in string:
            return string.split('Submission')[1].split(';')[0]

In [5]:
df = pd.read_csv('./data/input/sample_data.csv')
df['Submitted_by'] = df['Acc'].progress_apply(find_submited_by)
df['ID'] = df['Acc'].progress_apply(find_ID)
df['Biosample_Submission'] = df['ID'].progress_apply(find_submission_biosample_field)

100%|██████████| 33/33 [00:23<00:00,  1.43it/s]


Reorder the columns 

In [6]:
df = df[['Acc', 'ID', 'Center Names', 'Submitted_by', 'Biosample_Submission']]


In [7]:
df.head()

,Acc,ID,Center Names,Submitted_by,Biosample_Submission
0,SRR8606903,7314038,AUSTRALIAN INSTITUTE OF MARINCE SCIENCES,Australian Institute of Marince sciences,"Australian Institute of Marince sciences, Patr..."
1,SRR11788653,10848896,CLARK UNIVERSITY,Clark University,"Clark University, Nathan Ahlgren"
2,SRR9158324,7959983,CALIFORNIA STATE UNIVERSITY FULLERTON,California State University Fullerton,"California State University Fullerton, Maria R..."
3,SRR11881309,10964535,"CALIFORNIA STATE UNIVERSITY, FULLERTON","California State University, Fullerton","California State University, Fullerton, Joshua..."
4,SRR10293930,9209919,"CALIFORNIA STATE UNIVERSITY, LOS ANGELES","California State University, Los Angeles","California State University, Los Angeles, Kirs..."


In [8]:
df.to_csv('./data/output/sample_data_Scrape_Result.csv', encoding='utf-8')
